In [1]:
import pandas as pd
import geopandas as gpd
import openpyxl
import csv
import requests
from bs4 import BeautifulSoup
import time
import pytz
from shapely.geometry import Point
from datetime import datetime, timedelta
from keplergl import KeplerGl
from pyproj import Proj
import pyproj
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster
from tqdm import tqdm
import os, datetime, glob, re, json, ast, requests
import xml.etree.ElementTree as ET

### Load Data

#### 中央氣象局的資料(各縣市各月份雨量)

##### XML解析與轉檔

In [2]:
path = r"D:\Chu's Document!\02 Project\06 道路塌陷防治專案(天坑)\03 Data\Raw\雨量\*"
[x.split("雨量\\")[-1] for x in sorted(glob.glob(path))]

['2.5km解析度月氣候資料面化圖-每月雨量面化圖.json',
 'CWB雨量_xml.7z',
 'dy_Report_2014.xml',
 'dy_Report_2015.xml',
 'dy_Report_2016.xml',
 'dy_Report_2017.xml',
 'dy_Report_2018.xml',
 'dy_Report_2019.xml',
 'dy_Report_2020.xml',
 'dy_Report_2021.xml',
 'dy_Report_2022.xml',
 'dy_Report_2023.xml',
 'file.csv',
 'manifest.csv',
 'schema-file.csv',
 '氣象測站基本資料-有人氣象測站基本資料.json',
 '氣象測站基本資料-無人氣象測站基本資料.json',
 '臺北市水利處歷史日雨量(UTF8).csv',
 '臺北市水利處雨量站(csv).csv',
 '過去9年地面測站每日雨量資料.zip']

In [27]:
# 解析XML文件
xml_file_path = r"D:\Chu's Document!\02 Project\06 道路塌陷防治專案(天坑)\03 Data\Raw\雨量\dy_Report_2023.xml"
tree = ET.parse(xml_file_path)
root = tree.getroot()

# 打印根元素的標籤和屬性
print(f"Root Element: {root.tag}")
print(f"Attributes: {root.attrib}")

Root Element: {urn:cwa:gov:tw:cwacommon:0.1}cwaopendata
Attributes: {}


In [8]:
# 找到該筆資料的config
# start_time = root[10][0][0][6][0].text
# end_time = root[10][0][0][6][1].text
# col_name = root[10][0][0][8][1][1][3][0][1].text
# unit = root[10][0][0][8][0][2].text

# ch_name = root[10][0][1][0][0][0][1].text
# station_id = root[10][0][1][0][0][0][0].text
# en_name = root[10][0][1][0][0][0][2].text
# attr = root[10][0][1][0][0][0][3].text

# location_range = (0:35)
# root[10][0][1][0][0][i]
# year_month = (0:12)
# root[10][0][1][0][0][2][0][i]
# year_month = root[10][0][1][0][0][2][0][i][0]
# total_rain = root[10][0][1][0][0][2][0][i][1]
# root[10][0][1][0][0][2][0][1]

# root[10][0][1][0][1][2].tag
root[10][0][1][0][0][2][0][0][1].tag

'{urn:cwa:gov:tw:cwacommon:0.1}Total'

In [12]:
# 尋找location下的元素數量
# 定義命名空間
namespace = {'ns': 'urn:cwa:gov:tw:cwacommon:0.1'}

# 使用 XPath 表達式查詢特定層級的節點，加入命名空間前綴
location_nodes = root.findall('.//ns:location', namespace)
obstime_nodes = root.findall('.//ns:stationObsTimes', namespace)
# 獲取節點數量
num_location_nodes = len(location_nodes) 
num_obstime_nodes = len(obstime_nodes)
num_obstime_nodes

34

In [28]:
# 月雨量
cwb_data = pd.DataFrame()
# 迭代每個站點
for sta in range(0, num_location_nodes):
    ch_name = root[10][0][1][0][sta][0][1].text
    station_id = root[10][0][1][0][sta][0][0].text
    en_name = root[10][0][1][0][sta][0][2].text
    attr = root[10][0][1][0][sta][0][3].text
    date = root[10][0][1][0][0][1][0][0].text
    precipitation = root[10][0][1][0][0][1][0][1][0].text

    # 迭代每個月份
    for b in range(0, 12):
        year_month = root[10][0][1][0][sta][2][0][b][0].text
        total_rain = root[10][0][1][0][sta][2][0][b][1].text

         # 將資料添加到 DataFrame 中
        df_temp = pd.DataFrame({
            "ch_name": ch_name,
            "station_id": station_id,
            "en_name": en_name,
            "attr": attr,
            "year_month": year_month,
            "total_rain": total_rain
        }, index=[0])  # 注意這裡使用 index=[0] 來確保是一行

        # 使用 concat 合併資料
        cwb_data = pd.concat([cwb_data, df_temp], ignore_index=True)


# 顯示 DataFrame
cwb_data.head()

,ch_name,station_id,en_name,attr,year_month,total_rain
0,新北,466881,New Taipei,署屬有人氣象站,2023-01,52.0
1,新北,466881,New Taipei,署屬有人氣象站,2023-02,46.5
2,新北,466881,New Taipei,署屬有人氣象站,2023-03,35.5
3,新北,466881,New Taipei,署屬有人氣象站,2023-04,118.0
4,新北,466881,New Taipei,署屬有人氣象站,2023-05,278.0


In [47]:
date = root[10][0][1][0][0][1][1][0].text
precipitation = root[10][0][1][0][4][1][0][1][0].text
precipitation

'0.0'

In [16]:
# 日雨量
cwb_data = pd.DataFrame()
# 迭代每個站點
for sta in range(0, num_location_nodes):
    ch_name = root[10][0][1][0][sta][0][1].text
    station_id = root[10][0][1][0][sta][0][0].text
    en_name = root[10][0][1][0][sta][0][2].text
    attr = root[10][0][1][0][sta][0][3].text
    

    # 迭代每一天
    for b in range(0, 365):
        date = root[10][0][1][0][sta][1][b][0].text
        precipitation = root[10][0][1][0][sta][1][b][1][0].text

         # 將資料添加到 DataFrame 中
        df_temp = pd.DataFrame({
            "ch_name": ch_name,
            "station_id": station_id,
            "en_name": en_name,
            "attr": attr,
            "date": date,
            "precipitation": precipitation
        }, index=[0])  # 注意這裡使用 index=[0] 來確保是一行

        # 使用 concat 合併資料
        cwb_data = pd.concat([cwb_data, df_temp], ignore_index=True)


# 顯示 DataFrame
cwb_data.head()

,ch_name,station_id,en_name,attr,date,precipitation
0,新北,466881,New Taipei,署屬有人氣象站,2023-01-01,1.0
1,新北,466881,New Taipei,署屬有人氣象站,2023-01-02,6.0
2,新北,466881,New Taipei,署屬有人氣象站,2023-01-03,7.0
3,新北,466881,New Taipei,署屬有人氣象站,2023-01-04,0.5
4,新北,466881,New Taipei,署屬有人氣象站,2023-01-05,2.5


In [17]:
# 輸出為excel與csv
cwb_data.to_csv(r"D:\Chu's Document!\02 Project\06 道路塌陷防治專案(天坑)\03 Data\Processed\過程資料\cwb_daily_precipitation_2023.csv")
cwb_data.to_excel(r"D:\Chu's Document!\02 Project\06 道路塌陷防治專案(天坑)\03 Data\Processed\過程資料\cwb_daily_precipitation_2023.xlsx")

##### 108-112資料合併與轉日期格式

In [29]:
# Load
rain_2019 = pd.read_csv(r"D:\Chu's Document!\02 Project\06 道路塌陷防治專案(天坑)\03 Data\Processed\過程資料\cwb_monthly_precipitation_2019.csv")
rain_2020 = pd.read_csv(r"D:\Chu's Document!\02 Project\06 道路塌陷防治專案(天坑)\03 Data\Processed\過程資料\cwb_monthly_precipitation_2020.csv")
rain_2021 = pd.read_csv(r"D:\Chu's Document!\02 Project\06 道路塌陷防治專案(天坑)\03 Data\Processed\過程資料\cwb_monthly_precipitation_2021.csv")
rain_2022 = pd.read_csv(r"D:\Chu's Document!\02 Project\06 道路塌陷防治專案(天坑)\03 Data\Processed\過程資料\cwb_monthly_precipitation_2022.csv")
rain_2023 = pd.read_csv(r"D:\Chu's Document!\02 Project\06 道路塌陷防治專案(天坑)\03 Data\Processed\過程資料\cwb_monthly_precipitation_2023.csv")

In [30]:
data_list = [rain_2019, rain_2020, rain_2021, rain_2022, rain_2023]
rain_all = pd.concat(data_list)
rain_all.head()

,Unnamed: 0,ch_name,station_id,en_name,attr,year_month,total_rain
0,0,板橋,466880,BANQIAO,署屬有人氣象站,2019-01,48.8
1,1,板橋,466880,BANQIAO,署屬有人氣象站,2019-02,69.1
2,2,板橋,466880,BANQIAO,署屬有人氣象站,2019-03,197.8
3,3,板橋,466880,BANQIAO,署屬有人氣象站,2019-04,115.4
4,4,板橋,466880,BANQIAO,署屬有人氣象站,2019-05,388.7


In [31]:
# 去除不需要的欄位
rain_all = rain_all.drop(columns="Unnamed: 0")

# 將year-month欄位轉換為日期格式
rain_all["year_month"] = pd.to_datetime(rain_all["year_month"], format='%Y-%m')

# 將date欄位轉換為日期格式
# rain_all["date"] = pd.to_datetime(rain_all["date"], format='%Y-%m-%d')

In [32]:
rain_all.head()
rain_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2028 entries, 0 to 407
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   ch_name     2028 non-null   object        
 1   station_id  2028 non-null   object        
 2   en_name     2028 non-null   object        
 3   attr        2028 non-null   object        
 4   year_month  2028 non-null   datetime64[ns]
 5   total_rain  2005 non-null   object        
dtypes: datetime64[ns](1), object(5)
memory usage: 110.9+ KB


##### 108-112加入站點經緯度

In [22]:
# Load
location_people_json = pd.read_json(r"D:\Chu's Document!\02 Project\06 道路塌陷防治專案(天坑)\03 Data\Raw\雨量\氣象測站基本資料-有人氣象測站基本資料.json")
location_auto_json = pd.read_json(r"D:\Chu's Document!\02 Project\06 道路塌陷防治專案(天坑)\03 Data\Raw\雨量\氣象測站基本資料-有人氣象測站基本資料.json")


In [23]:
# 取出站點資料
# 有人氣象站
location_people = pd.DataFrame(location_people_json["cwaopendata"][9]["resource"]["data"]["stationsStatus"]["station"])
# location_people.head(2)

# 無人氣象站
location_auto = pd.DataFrame(location_auto_json["cwaopendata"][9]["resource"]["data"]["stationsStatus"]["station"])
location_auto.head()

,StationID,StationName,StationNameEN,StationAltitude,StationLongitude,StationLatitude,CountyName,Location,StationStartDate,StationEndDate,status,Notes,OriginalStationID,NewStationID
0,466850,五分山雷達站,WUFENSHAN,756.0,121.781205,25.071182,新北市,瑞芳區靜安路四段1巷1號,1988-07-01,None,現存測站,本站只有雷達觀測資料。,None,None
1,466880,板橋,BANQIAO,9.7,121.442017,24.997647,新北市,板橋區大觀路二段265巷62號,2002-01-01,2022-12-31,已撤銷,原為探空站，自2002年開始進行氣象觀測。因受附近興建合宜住宅影響，日照觀測自民國104年起...,None,466881
2,466881,新北,New Taipei,24.1,121.520000,24.959300,新北市,新店區莒光路29號,2023-01-01,None,現存測站,為臺北氣象站板橋站區(站碼466880)遷移之新站，於2023/01/03上午11時取代舊站...,466880,None
3,466900,淡水,TAMSUI,19.0,121.448906,25.164889,新北市,淡水區中正東路42巷6號,1942-10-01,None,現存測站,None,None,None
4,466910,鞍部,ANBU,837.6,121.529731,25.182586,臺北市,北投區陽明山竹子湖路111號,1943-01-01,None,現存測站,None,None,None


In [130]:
# 確認二筆資料的站點是否一致(可能有些有人有些無人)
if location_people["StationName"].any() == location_auto["StationName"].any():
    print("完全一致")
else:
    print("不一致")

完全一致


In [33]:
# 依照StationID加入lng, lat
keep_col = ["StationID", "StationLongitude", "StationLatitude"]
df_loca = location_auto[keep_col]

# 重新命名columns
rename_col = {"StationID":"station_id", "StationLongitude":"station_lng", "StationLatitude":"station_lat"}
df_loca = df_loca.rename(columns=rename_col)

# # 合併欄位
rain_all = pd.merge(rain_all, df_loca, on='station_id', how='left')
rain_all.head()

,ch_name,station_id,en_name,attr,year_month,total_rain,station_lng,station_lat
0,板橋,466880,BANQIAO,署屬有人氣象站,2019-01-01,48.8,121.442017,24.997647
1,板橋,466880,BANQIAO,署屬有人氣象站,2019-02-01,69.1,121.442017,24.997647
2,板橋,466880,BANQIAO,署屬有人氣象站,2019-03-01,197.8,121.442017,24.997647
3,板橋,466880,BANQIAO,署屬有人氣象站,2019-04-01,115.4,121.442017,24.997647
4,板橋,466880,BANQIAO,署屬有人氣象站,2019-05-01,388.7,121.442017,24.997647


In [35]:
rain_all.to_excel(r"D:\Chu's Document!\02 Project\06 道路塌陷防治專案(天坑)\03 Data\Processed\cwb_monthly_precipitation_2019-2023.xlsx")
rain_all.to_csv(r"D:\Chu's Document!\02 Project\06 道路塌陷防治專案(天坑)\03 Data\Processed\cwb_monthly_precipitation_2019-2023.csv")

##### 嘗試過程記錄

In [32]:
cwb_data["ch_name"].unique()

array(['板橋', '淡水', '鞍部', '臺北', '竹子湖', '基隆', '彭佳嶼', '花蓮', '新屋', '蘇澳', '宜蘭',
       '金門', '田中', '東吉島', '澎湖', '臺南', '高雄', '嘉義', '臺中', '阿里山', '大武', '玉山',
       '新竹', '恆春', '成功', '蘭嶼', '日月潭', '臺東', '馬祖', '桃園', '彰化', '雲林', '屏東',
       '苗栗'], dtype=object)

In [35]:
root.find("{urn:cwa:gov:tw:cwacommon:0.1}resources")

<Element '{urn:cwa:gov:tw:cwacommon:0.1}resources' at 0x0000026EF8F51940>

In [24]:
# 取得所有層級的標籤
all_tags = [elem.tag for elem in root.iter()]

# 印出標籤列表和數量
print(f"所有層級的標籤： {all_tags}")
print(f"總共有 {len(all_tags)} 個標籤。")

所有層級的標籤： ['{urn:cwa:gov:tw:cwacommon:0.1}cwaopendata', '{urn:cwa:gov:tw:cwacommon:0.1}identifier', '{urn:cwa:gov:tw:cwacommon:0.1}datasetID', '{urn:cwa:gov:tw:cwacommon:0.1}datasetName', '{urn:cwa:gov:tw:cwacommon:0.1}dataItemID', '{urn:cwa:gov:tw:cwacommon:0.1}sender', '{urn:cwa:gov:tw:cwacommon:0.1}sent', '{urn:cwa:gov:tw:cwacommon:0.1}status', '{urn:cwa:gov:tw:cwacommon:0.1}scope', '{urn:cwa:gov:tw:cwacommon:0.1}msgType', '{urn:cwa:gov:tw:cwacommon:0.1}publisherOID', '{urn:cwa:gov:tw:cwacommon:0.1}resources', '{urn:cwa:gov:tw:cwacommon:0.1}resource', '{urn:cwa:gov:tw:cwacommon:0.1}metadata', '{urn:cwa:gov:tw:cwacommon:0.1}resourceID', '{urn:cwa:gov:tw:cwacommon:0.1}resourceName', '{urn:cwa:gov:tw:cwacommon:0.1}resourceDescription', '{urn:cwa:gov:tw:cwacommon:0.1}language', '{urn:cwa:gov:tw:cwacommon:0.1}homepage', '{urn:cwa:gov:tw:cwacommon:0.1}sources', '{urn:cwa:gov:tw:cwacommon:0.1}source', '{urn:cwa:gov:tw:cwacommon:0.1}title', '{urn:cwa:gov:tw:cwacommon:0.1}path', '{urn:cwa:gov

In [19]:
root[10][0][1][0][0].tag

'{urn:cwa:gov:tw:cwacommon:0.1}location'

In [46]:
station_data = root[10][0][1][0][0]

for item in station_data.findall("station"):
    id = item.find("StationID").text
    ch_name = item.find("StationName").text
    en_name = item.find("StationNameEN").text
    attr = item.find("StationAttribute").text
    print(id, ch_name)

[]


In [12]:
# 遍歷子元素
for child in root:
    print(f"Child Element: {child.tag}, Attributes: {child.attrib}")

    # 遍歷孫元素
    for grandchild in child:
        print(f"Grandchild Element: {grandchild.tag}, Text: {grandchild.text}")

Child Element: {urn:cwa:gov:tw:cwacommon:0.1}identifier, Attributes: {}
Child Element: {urn:cwa:gov:tw:cwacommon:0.1}datasetID, Attributes: {}
Child Element: {urn:cwa:gov:tw:cwacommon:0.1}datasetName, Attributes: {}
Child Element: {urn:cwa:gov:tw:cwacommon:0.1}dataItemID, Attributes: {}
Child Element: {urn:cwa:gov:tw:cwacommon:0.1}sender, Attributes: {}
Child Element: {urn:cwa:gov:tw:cwacommon:0.1}sent, Attributes: {}
Child Element: {urn:cwa:gov:tw:cwacommon:0.1}status, Attributes: {}
Child Element: {urn:cwa:gov:tw:cwacommon:0.1}scope, Attributes: {}
Child Element: {urn:cwa:gov:tw:cwacommon:0.1}msgType, Attributes: {}
Child Element: {urn:cwa:gov:tw:cwacommon:0.1}publisherOID, Attributes: {}
Child Element: {urn:cwa:gov:tw:cwacommon:0.1}resources, Attributes: {}
Grandchild Element: {urn:cwa:gov:tw:cwacommon:0.1}resource, Text: 
            


In [8]:
data_2022 = pd.read_xml(r"D:\Chu's Document!\02 Project\06 道路塌陷防治專案(天坑)\03 Data\Raw\雨量\dy_Report_2022.xml")
data_2022.head()


,identifier,datasetID,datasetName,dataItemID,sender,sent,status,scope,msgType,publisherOID,resource
0,b7b497f3-7423-455c-8816-7cf95943588a,None,None,None,None,None,None,None,None,None,NaN
1,None,S0001,None,None,None,None,None,None,None,None,NaN
2,None,None,過去氣象觀測,None,None,None,None,None,None,None,NaN
3,None,None,None,S0001-01000,None,None,None,None,None,None,NaN
4,None,None,None,None,od@cwa.gov.tw,None,None,None,None,None,NaN


#### 臺北市工務局水利處資料(臺北市各日歷史雨量及雨量站)

##### Load Data

In [102]:
# file path
path_01 = r"D:\TUIC Projects\02 Project\06 道路塌陷防治專案(天坑)\03 Data\Raw\臺北市水利處雨量站(csv).csv"
path_02 = r"D:\TUIC Projects\02 Project\06 道路塌陷防治專案(天坑)\03 Data\Raw\臺北市工務局水利處_歷史日雨量(UTF8).csv"

# load
station_raw = pd.read_csv(path_01)
precipitation_raw = pd.read_csv(path_02)

station = station_raw.copy()
precipitation = precipitation_raw.copy()

In [107]:
# 查看原始資料
print(station.head())
print(f"資料大小{station.shape}")
print(station.isna().sum())

  TOWN STATIONID  NAME          經度         緯度
0  北投區     '001'  湖田國小  121.538950  25.168698
1  北投區     '002'  大屯國小  121.504080  25.155133
2  北投區     '003'  桃源國中  121.476976  25.134551
3  北投區     '004'  北投國小  121.499824  25.133619
4  士林區     '005'  陽明高中  121.516857  25.092660
資料大小(42, 5)
TOWN         0
STATIONID    0
NAME         0
經度           0
緯度           0
dtype: int64


In [109]:
# 查看原始資料
print(precipitation.head(2))
print(f"資料大小{precipitation.shape}")
print(precipitation.isna().sum())

     站名      年月  1日雨量  2日雨量  3日雨量  4日雨量  5日雨量  6日雨量  7日雨量  8日雨量  ...  22日雨量  \
0  三興國小  200007   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...    0.0   
1  大屯國小  200007   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...    0.0   

   23日雨量  24日雨量  25日雨量  26日雨量  27日雨量  28日雨量  29日雨量  30日雨量  31日雨量  
0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  
1    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  

[2 rows x 33 columns]
資料大小(8810, 33)
站名       0
年月       0
1日雨量     0
2日雨量     0
3日雨量     0
4日雨量     0
5日雨量     0
6日雨量     0
7日雨量     0
8日雨量     0
9日雨量     0
10日雨量    0
11日雨量    0
12日雨量    0
13日雨量    0
14日雨量    0
15日雨量    0
16日雨量    0
17日雨量    0
18日雨量    0
19日雨量    0
20日雨量    0
21日雨量    0
22日雨量    0
23日雨量    0
24日雨量    0
25日雨量    0
26日雨量    0
27日雨量    0
28日雨量    0
29日雨量    0
30日雨量    0
31日雨量    0
dtype: int64


In [112]:
precipitation["年月"].max()

202306

##### 統計歷史各月雨量並串接站點經緯度